### In this notebook i gather datasets and prepare ML model

Read dataset from other notebooks

In [9]:
%store -r df_rel
%store -r data

In [10]:
df_rel

,Białystok,Bydgoszcz,Gdańsk,Gdynia,Katowice,Kielce,Kraków,Lublin,Łódź,Olsztyn,Opole,Poznań,Rzeszów,Szczecin,Warszawa,Wrocław,Zielona Góra
date,,,,,,,,,,,,,,,,,
2006-10-01,0.000000,704.000000,1195.000000,347.000000,76.040000,573.973146,1908.000000,-142.00,1027.000000,800.000,1196.067918,57.294085,539.000000,464.142700,222.000000,1009.000000,115.021741
2007-01-01,1454.000000,554.000000,238.000000,1182.000000,456.060000,-110.898634,318.000000,290.00,690.000000,349.000,682.689077,2447.518593,19.000000,1070.000985,1414.000000,549.702547,2.641748
2007-04-01,544.000000,1199.000000,1562.000000,2468.000000,662.190000,519.859354,-118.000000,539.00,526.000000,-373.000,645.813428,85.272568,660.000000,300.845051,491.000000,-14.702547,1281.550423
2007-07-01,-526.000000,-706.000000,54.000000,-820.000000,1054.970000,749.478259,291.000000,502.00,-252.000000,124.000,-1083.269825,-522.257291,-11.000000,-15.248609,740.000000,150.287532,54.079986
2007-10-01,796.000000,-165.000000,-651.000000,-227.000000,222.010000,-175.900993,-65.000000,402.00,-139.000000,0.000,153.290446,725.948857,288.000000,696.415424,821.000000,-949.151222,-242.093085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01,533.237059,535.442361,875.644344,-250.255501,-2.588626,135.558069,389.963068,-68.01,111.744148,285.461,291.649629,418.846499,505.285525,383.068354,414.229635,223.753714,309.373237
2022-01-01,163.115943,206.318343,170.691943,400.164507,465.404052,355.001485,146.873607,273.09,27.801881,606.894,-308.799386,641.685724,325.859507,240.303475,697.535467,163.054800,166.134142
2022-04-01,293.020320,565.555774,276.445056,258.619892,445.640417,579.877269,238.548637,329.62,336.090987,150.888,1390.784945,127.119602,310.552066,483.521246,377.706850,436.841053,42.771298


In [11]:
data

,year,building_permissions,region,flats_sold
0,2005,153,MAŁOPOLSKIE,NaN
1,2006,274,MAŁOPOLSKIE,NaN
2,2007,342,MAŁOPOLSKIE,NaN
3,2008,253,MAŁOPOLSKIE,NaN
4,2009,178,MAŁOPOLSKIE,NaN
...,...,...,...,...
165,2017,112,WARMIŃSKO-MAZURSKIE,549.0
166,2018,126,WARMIŃSKO-MAZURSKIE,495.0
167,2019,129,WARMIŃSKO-MAZURSKIE,716.0
168,2020,107,WARMIŃSKO-MAZURSKIE,682.0
